Wzorowałam się na https://www.kaggle.com/bguberfain/naive-xgb-lb-0-317. Niestety moje próby pogorszyły wynik zamiast polepszyć, ale bardzo trudno było sprawdzać poprawność wyniku, gdyż w walidacji krzyżowej wyniki wychodziły dużo gorsze, a przede wszystkim zupełnie inne niż później na zbiorze testowym. Ma to zapewne związek z charakterem danych (szereg czasowy) i faktem, że dane testowe pochodzą z późniejszego okresu niż dane treningowe. Warto byłoby budować zbiór walidacyjnie w analogiczny sposób. W stosunku do powyższego notebooka zdecydowałam się zaimplementować cały proces przetwarzania i modelowania danych jako pipeline, który jest dużo czytelniejszym i łatwiejszym w późniejszym utrzymaniu sposobem.  

In [13]:
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy import sparse as sp
from sklearn.pipeline import make_pipeline, make_union
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer, OneHotEncoder
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer

In [14]:
train_df = pd.read_csv('train.csv',index_col="tripid")
df_test = pd.read_csv('test.csv',index_col="tripid")

In [15]:
train_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [17]:
train_df['checkout_datetime'] = pd.to_datetime(train_df['drop_time'])
train_df['checkin_datetime'] = pd.to_datetime(train_df['pickup_time'])
#train_df['duration'] = train_df['checkout_datetime']-train_df['checkin_datetime']
train_df['duration_in_minutes'] = (train_df['checkout_datetime']-train_df['checkin_datetime'])/pd.Timedelta(minutes=1)
train_df['lat']  = abs(train_df['pick_lat']-train_df['drop_lat'])
train_df['lon']  = abs(train_df['pick_lon']-train_df['drop_lon'])
#train_df['distance'] = (((train_df['lat'])**2 + (train_df['lon'])**2)**(1/2))*100
#train_df['manhatton_distance'] = (train_df['lat'] + train_df['lat'])

#train_df.loc[train_df['meter_waiting_fare'] > 0, 'is_meter_waiting_fare'] = 1
#train_df.loc[train_df['meter_waiting_fare'] <= 0, 'is_meter_waiting_fare'] = 0

#train_df['meter_waiting_fare'] = train_df['meter_waiting_fare'].astype(np.int64)


#train_df['is_waiting_fare'] = train_df['meter_waiting']*train_df['meter_waiting_fare']
#train_df['waiting_till_pickup_fare'] = train_df['meter_waiting_till_pickup']*train_df['meter_waiting_fare']

train_df['date_time'] = pd.to_datetime(train_df['drop_time'])
train_df['drop_Hour'] = train_df['date_time'].apply(lambda x: x.hour+1)
#train_df['weekday'] = train_df['date_time'].apply(lambda x: x.weekday())
train_df['remaining_fare'] = train_df['fare']-train_df['additional_fare']-train_df['meter_waiting_fare']
#train_df['r_fare'] = train_df['fare']-train_df['additional_fare']
train_df['mobile_time'] = train_df['duration']-train_df['meter_waiting']-train_df['meter_waiting_till_pickup']
#train_df['m_time'] = train_df['duration']-train_df['meter_waiting']
#train_df['unknown'] = train_df['duration']-train_df['duration_in_minutes']*60
#train_df['mobile_fare_time'] = train_df['remaining_fare']-train_df['mobile_time']
#train_df['unknown_new'] = (train_df['lat']**2)+(train_df['lon']**2)

#train_df
#train_df['pick_Hour'] = pd.to_datetime(train_df['pickup_time'])
#train_df['pick_Hour'] = train_df['pick_Hour'].apply(lambda x: x.hour+1)

#features_df_num = train_df.drop(["date_time","pickup_time","drop_time","duration","checkin_datetime","drop_lat","drop_lon","checkout_datetime", "label"], axis = 1)
features_df = train_df.drop(["date_time","duration","pickup_time","drop_time","checkin_datetime","drop_lat","drop_lon","pick_lat","pick_lon","checkout_datetime", "label"], axis = 1)

pure_train_df = train_df.drop(["label"], axis = 1)
categorical_features = pure_train_df.columns[pure_train_df.dtypes == "object"].values


label_df = train_df[["label"]]
label_df = label_df.replace({'label': {'correct': 1, 'incorrect': 0}})

features_df.head()

,additional_fare,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,duration_in_minutes,lat,lon,drop_Hour,remaining_fare,mobile_time
tripid,,,,,,,,,,,
189123628,10.5,56.0,0.0000,64.0,270.32,14.0,0.04078,0.0210,1,259.8200,714.0
189125358,10.5,47.0,0.0000,134.0,197.85,13.0,0.02784,0.0061,2,187.3500,610.0
189125719,10.5,80.0,0.0000,61.0,301.64,18.0,0.02830,0.0495,2,291.1400,946.0
189127273,10.5,271.0,15.6638,68.0,82.30,10.0,0.00178,0.0076,3,56.1362,259.0
189128020,NaN,NaN,NaN,NaN,358.39,17.0,0.02963,0.0675,4,NaN,NaN


In [18]:
# Selector coumns by name or type
class PandasSelector(BaseEstimator, TransformerMixin):
    def __init__(self, dtype=None, columns=None, inverse=False,
                 return_vector=True):
        self.dtype = dtype
        self.columns = columns
        self.inverse = inverse
        self.return_vector = return_vector

    def check_condition(self, x, col):
        cond = (self.dtype is not None and x[col].dtype == self.dtype) or \
               (self.columns is not None and col in self.columns)
        return self.inverse ^ cond

    def fit(self, x, y=None):
        return self

    def _check_if_all_columns_present(self, x):
        if not self.inverse and self.columns is not None:
            missing_columns = set(self.columns) - set(x.columns)
            if len(missing_columns) > 0:
                missing_columns_ = ','.join(col for col in missing_columns)
                raise KeyError('Keys are missing in the record: %s' %
                               missing_columns_)

    def transform(self, x):
        # check if x is a pandas DataFrame
        if not isinstance(x, pd.DataFrame):
            raise KeyError('Input is not a pandas DataFrame')

        selected_cols = []
        for col in x.columns:
            if self.check_condition(x, col):
                selected_cols.append(col)

        # if the column was selected and inversed = False make sure the column
        # is in the DataFrame
        self._check_if_all_columns_present(x)

        # if only 1 column is returned return a vector instead of a dataframe
        if len(selected_cols) == 1 and self.return_vector:
            return np.array(x[selected_cols[0]])
        else:
            return np.array(x[selected_cols])

In [19]:
# Converter fron string to int (for one hot encoder)
class StringConverter(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.map = {} # column : string : int
    
    def fit(self, X, *args):
        for col in range(X.shape[1]):
            self.map[col] = {}
            idx = 1
            for row in range(X.shape[0]):                
                s = X[row, col]
                if s not in self.map[col]:
                    self.map[col][s] = idx
                    idx += 1
        return self

    def transform(self, X):
        X_int = np.zeros(shape=X.shape)
        for col in range(X.shape[1]):
            X_int[:, col] = np.array([self.map[col].get(s, 0) for s in X[:, col]])

        return X_int

In [20]:
# Adds a column in sparse matrix (because of bug in xgboost)
class AddDummy(BaseEstimator, TransformerMixin):
    def fit(self, X, *args):
        return self
    
    def transform(self, X):
        return sp.hstack([X, sp.csr_matrix(np.ones((X.shape[0], 1)))])

In [21]:
# Adds features based on date
class DatesFeaturer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.month_year_map = {}
        self.week_year_map = {}
    
    def fit(self, df, *args):
        month_year = pd.to_datetime(df.timestamp).dt.month + pd.to_datetime(df.timestamp).dt.year * 100
        self.month_year_map = month_year.value_counts().to_dict()
        week_year = pd.to_datetime(df.timestamp).dt.weekofyear + pd.to_datetime(df.timestamp).dt.year * 100
        self.week_year_map = week_year.value_counts().to_dict()
        return self
    
    def transform(self, df):
        month_year = pd.to_datetime(df.timestamp).dt.month + pd.to_datetime(df.timestamp).dt.year * 100
        week_year = pd.to_datetime(df.timestamp).dt.weekofyear + pd.to_datetime(df.timestamp).dt.year * 100
        
        new_df = pd.DataFrame({
            'month_year_count': month_year.map(self.month_year_map),
            'week_year_count': week_year.map(self.week_year_map),
            'month': pd.to_datetime(df.timestamp).dt.month,
            'dow': pd.to_datetime(df.timestamp).dt.dayofweek
        })
        
        return np.array(new_df)

In [22]:
# Adds relative features
class RelativeFeaturer(BaseEstimator, TransformerMixin):
    
    def fit(self, df, *args):
        return self
    
    def transform(self, df):   
        new_df = pd.DataFrame({
            'rel_floor': df['floor'] / np.maximum(1.0, df['max_floor'].astype(float)),
            'rel_kitch_sq': df['kitch_sq'] / np.minimum(1.0, df['full_sq'].astype(float)),
        })
        return np.array(new_df)

In [23]:
# Converts estimator to transform in order to ensemble many estimators
class EstimatorToTransform(BaseEstimator, TransformerMixin):    
    def __init__(self, estimator):
        self.estimator = estimator
    
    def fit(self, X, *args):
        self.estimator.fit(X, *args)
        return self

    def transform(self, X):
        pred = self.estimator.predict(X)
        return pred.reshape(-1, 1)

In [24]:
def rmsle_score(pred, true):
    return (np.sum((np.log(1 + pred) - np.log(1 + true))**2) / len(pred))**0.5

In [25]:
# Defining pipelines

float_pipeline = make_pipeline(
    PandasSelector(dtype=float),
)

eco_pipeline = make_pipeline(
    PandasSelector(columns=['ecology'], return_vector=False),
    StringConverter(),
    OneHotEncoder(),
)

prod_pipeline = make_pipeline(
    PandasSelector(columns=['product_type'], return_vector=False),
    StringConverter(),
    OneHotEncoder(),
)

sub_pipeline = make_pipeline(
    PandasSelector(columns=['sub_area'], return_vector=False),
    StringConverter(),
    OneHotEncoder(),
)

dates_pipeline = make_pipeline(
    DatesFeaturer(),
)

rel_pipeline = make_pipeline(
    RelativeFeaturer(),
)

pipeline_ensemble = make_pipeline(
    make_union(
        EstimatorToTransform(
            xgb.XGBRegressor(
                n_estimators=500,
                learning_rate=0.05,
                max_depth=5,
                subsample=0.7,
                colsample_bytree=0.7,
                objective='reg:linear',
            ),
        ),
        EstimatorToTransform(
            xgb.XGBRegressor(
                n_estimators=100,
                learning_rate=0.05,
                max_depth=3,
                subsample=1,
                colsample_bytree=1,
                objective='reg:linear',
            ),
        ),
    ),
)

final_pipeline = make_pipeline(
    make_union(
        rel_pipeline,
        dates_pipeline,
        float_pipeline,
        eco_pipeline,
        prod_pipeline,
        sub_pipeline,
    ),
    AddDummy(),
    Imputer(),
    pipeline_ensemble,
    LinearRegression(),

)

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [28]:
# Calculate cross-validate score

cv_score = cross_val_score(
        final_pipeline,
        features_df,
        label_df,
        scoring=make_scorer(rmsle_score),
        cv=3,
)

np.mean(cv_score)

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


KeyError: 'floor'

In [15]:
%%time
# Fitting model
model = final_pipeline.fit(x_train, y_train)

CPU times: user 6min 3s, sys: 5min 58s, total: 12min 1s
Wall time: 6min 29s


In [16]:
pred_train = model.predict(x_train)
print(rmsle_score(pred_train, y_train))

0.40290897149012106


/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app


In [17]:
# Predicting
final_pred = model.predict(x_final)

In [18]:
# Creating final submission file
df_sub = pd.DataFrame({'id': id_test.astype(int), 'price_doc': final_pred.astype(int)})
df_sub.to_csv('sub.csv', index=False)